In [8]:
import pandas as pd

hist_url = "https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv"
df = pd.read_csv(hist_url)
# helpful columns: USAF, WBAN, STATION NAME, CTRY, ICAO, LAT, LON, ELEV(M), BEGIN, END

mask_country = df['CTRY'].eq('KG')
mask_name = df['STATION NAME'].str.contains('BISH|MANAS|FRUNZE', case=False, na=False)

kg_stations = df[mask_country & mask_name].copy()
print(kg_stations['USAF'].unique())


['382200' '383530' '383531']


In [9]:
inv_url = "https://www.ncei.noaa.gov/pub/data/noaa/isd-inventory.csv"
inv = pd.read_csv(inv_url)

# pick a USAF/WBAN you found above, for example:
USAF = "383531"    # e.g., '389830'
WBAN = "99999"    # e.g., '99999'

have = inv[(inv['USAF'].astype(str).str.zfill(6)==str(USAF).zfill(6)) &
           (inv['WBAN'].astype(str).str.zfill(5)==str(WBAN).zfill(5))]
print(have.head())      # shows YEAR, MONTH, and record counts


         USAF   WBAN  YEAR   JAN   FEB   MAR   APR   MAY   JUN   JUL   AUG  \
25225  383531  99999  2020  1478  1387  1463  1408  1483  1433  1482  1483   
25226  383531  99999  2021  1488  1093  1485  1438  1487  1440  1488  1488   
25227  383531  99999  2022  1488  1343  1485  1432  1482  1435  1487  1470   
25228  383531  99999  2023  1484  1328  1464  1436  1470  1402  1472  1457   
25229  383531  99999  2024  1486  1388  1253  1367  1428  1311  1439  1422   

        SEP   OCT   NOV   DEC  
25225  1439  1486  1431  1488  
25226  1439  1488  1440  1482  
25227  1431  1452  1436  1452  
25228  1391  1469  1427  1486  
25229  1411  1486  1238  1423  


/var/folders/83/t702v3093vj_m1cq21gyvzgh0000gn/T/ipykernel_79891/3537498492.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  inv = pd.read_csv(inv_url)


In [16]:
import urllib.request
import gzip
import io
import pandas as pd

# ---- CONFIG ----
USAF = "383530"      # <-- put your USAF here (6 digits, zero-padded if needed)
WBAN = "99999"       # <-- put your WBAN here (5 digits, often 99999 outside US)
START_YEAR = 2016
END_YEAR   = 2025    # inclusive
SAVE_CSV   = f"isd_lite_{USAF}_{WBAN}_{START_YEAR}_{END_YEAR}.csv"

# ISD-Lite column layout (fixed-width), per NOAA docs
WIDTHS = [4, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6]
NAMES  = [
    "year", "month", "day", "hour",
    "air_temp_tenths_c", "dewpoint_tenths_c", "slp_tenths_hpa",
    "wind_dir_deg", "wind_speed_tenths_ms",
    "sky_cover_code", "precip_1h_tenths_mm", "precip_6h_tenths_mm"
]

def fetch_isd_lite_year(usaf: str, wban: str, year: int) -> pd.DataFrame | None:
    """
    Download and parse one ISD-Lite file for a station-year.
    Returns a DataFrame or None if file is missing/unavailable.
    """
    url = f"https://www.ncei.noaa.gov/pub/data/noaa/isd-lite/{year}/{usaf}-{wban}-{year}.gz"
    try:
        with urllib.request.urlopen(url) as resp:
            # Decompress to text stream
            with gzip.GzipFile(fileobj=io.BytesIO(resp.read())) as gz:
                # Read fixed-width into DataFrame
                df = pd.read_fwf(gz, widths=WIDTHS, names=NAMES)
    except urllib.error.HTTPError as e:
        # Commonly 404 if that station-year doesn't exist
        print(f"[WARN] {year}: HTTP {e.code} for {url} — skipping.")
        return None
    except Exception as e:
        print(f"[WARN] {year}: Failed to fetch/parse — {e}")
        return None

    # Build datetime (assume UTC in ISD/ISD-Lite)
    try:
        dt = pd.to_datetime(df[['year','month','day','hour']], errors='coerce', utc=True)
        df.insert(0, 'datetime_utc', dt)
    except Exception as e:
        print(f"[WARN] {year}: datetime parse issue — {e}")

    # Replace missing sentinels and scale numeric fields
    # -9999 / -999 / etc. appear as missing; safest is to set < -9000 to NA for tenths fields
    for col in ["air_temp_tenths_c","dewpoint_tenths_c","slp_tenths_hpa",
                "wind_speed_tenths_ms","precip_1h_tenths_mm","precip_6h_tenths_mm"]:
        if col in df.columns:
            df[col] = df[col].where(df[col] > -9000, pd.NA)

    # Scaled columns
    df["air_temp_c"]     = df["air_temp_tenths_c"] / 10
    df["dewpoint_c"]     = df["dewpoint_tenths_c"] / 10
    df["slp_hpa"]        = df["slp_tenths_hpa"] / 10
    df["wind_speed_ms"]  = df["wind_speed_tenths_ms"] / 10
    df["precip_1h_mm"]   = df["precip_1h_tenths_mm"] / 10
    df["precip_6h_mm"]   = df["precip_6h_tenths_mm"] / 10

    # Local time (Asia/Bishkek, UTC+6)
    try:
        df["datetime_local"] = df["datetime_utc"].dt.tz_convert("Asia/Bishkek")
    except Exception:
        # If datetime_utc isn't tz-aware for any reason, localize first then convert
        df["datetime_local"] = (pd.to_datetime(df["datetime_utc"], utc=True)
                                .dt.tz_convert("Asia/Bishkek"))

    # Keep a tidy subset plus raw fields if you want them for QA/QC
    cols_order = [
        "datetime_utc", "datetime_local",
        "air_temp_c", "dewpoint_c", "slp_hpa", "wind_dir_deg", "wind_speed_ms",
        "sky_cover_code", "precip_1h_mm", "precip_6h_mm",
        "year","month","day","hour",  # originals for audit
    ]
    existing = [c for c in cols_order if c in df.columns]
    return df[existing].copy()

# ---- Fetch loop and combine ----
frames = []
for yr in range(START_YEAR, END_YEAR + 1):
    print(f"[INFO] Fetching {USAF}-{WBAN}-{yr} ...")
    df_year = fetch_isd_lite_year(USAF, WBAN, yr)
    if df_year is not None and not df_year.empty:
        frames.append(df_year)

if not frames:
    raise SystemExit("No data retrieved — check USAF/WBAN IDs or availability for those years.")

combined = pd.concat(frames, ignore_index=True).drop_duplicates(subset=["datetime_utc"])
combined = combined.sort_values("datetime_utc").reset_index(drop=True)

print(combined.head())
print(combined.tail())
print(f"[INFO] Combined shape: {combined.shape}")

# Save to CSV
combined.to_csv(SAVE_CSV, index=False)
print(f"[OK] Saved to {SAVE_CSV}")


[INFO] Fetching 383530-99999-2016 ...
[INFO] Fetching 383530-99999-2017 ...
[INFO] Fetching 383530-99999-2018 ...
[INFO] Fetching 383530-99999-2019 ...
[INFO] Fetching 383530-99999-2020 ...
[INFO] Fetching 383530-99999-2021 ...
[INFO] Fetching 383530-99999-2022 ...
[INFO] Fetching 383530-99999-2023 ...
[INFO] Fetching 383530-99999-2024 ...
[INFO] Fetching 383530-99999-2025 ...
               datetime_utc            datetime_local  air_temp_c  dewpoint_c  \
0 2016-01-01 00:00:00+00:00 2016-01-01 06:00:00+06:00         4.9        -5.8   
1 2016-01-01 03:00:00+00:00 2016-01-01 09:00:00+06:00         4.7        -3.7   
2 2016-01-01 06:00:00+00:00 2016-01-01 12:00:00+06:00        11.6        -2.2   
3 2016-01-01 09:00:00+00:00 2016-01-01 15:00:00+06:00        13.8        -2.2   
4 2016-01-01 12:00:00+00:00 2016-01-01 18:00:00+06:00         9.5        -3.3   

   slp_hpa  wind_dir_deg  wind_speed_ms  sky_cover_code  precip_1h_mm  \
0   1023.4           280            1.0               8     

In [17]:
combined.describe()

,air_temp_c,dewpoint_c,slp_hpa,wind_dir_deg,wind_speed_ms,sky_cover_code,precip_1h_mm,precip_6h_mm,year,month,day,hour
count,26495.000000,26465.000000,26304.000000,26532.000000,22147.000000,26532.000000,0.0,3.000000,26532.000000,26532.000000,26532.000000,26532.000000
mean,12.906243,2.589972,1017.827783,161.336876,1.472841,-1398.028155,NaN,3.666667,2020.357154,6.405548,15.700814,10.526082
std,11.714525,6.898117,10.446068,231.462566,1.942067,3473.869635,NaN,3.511885,2.797270,3.414833,8.790157,6.902611
min,-26.600000,-29.000000,994.500000,-9999.000000,0.000000,-9999.000000,NaN,0.000000,2016.000000,1.000000,1.000000,0.000000
25%,3.500000,-2.300000,1009.575000,40.000000,1.000000,1.000000,NaN,2.000000,2018.000000,3.000000,8.000000,6.000000
50%,13.600000,3.400000,1017.100000,180.000000,1.000000,5.000000,NaN,4.000000,2020.000000,6.000000,16.000000,9.000000
75%,22.200000,8.100000,1025.400000,270.000000,2.000000,8.000000,NaN,5.500000,2023.000000,9.000000,23.000000,18.000000
max,39.600000,20.700000,1058.900000,360.000000,53.000000,9.000000,NaN,7.000000,2025.000000,12.000000,31.000000,21.000000


# Combining PurpleAir data


In [ ]:
# df_4 = pd.read_csv('44919 2016-08-26 2025-08-26 60-Minute Average.csv')
# df_9 = pd.read_csv('92387 2016-08-26 2025-08-26 60-Minute Average.csv')
# df_2 = pd.read_csv('217883 2016-08-26 2025-08-26 60-Minute Average.csv')

# combined = pd.concat([df_4, df_9, df_4], ignore_index=True).drop_duplicates(subset=["time_stamp"])
# combined = combined.sort_values("datetime_utc").reset_index(drop=True)




,time_stamp,humidity,temperature,pressure,voc,analog_input,pm2.5_alt|pm2.5_alt = C * (0.00030418*N1 0.0018512*N2 0.02069706*N3),deciviews,visual_range,0.3_um_count,...,1.0_um_count,2.5_um_count,5.0_um_count,10.0_um_count,pm1.0_cf_1,pm1.0_atm,pm2.5_atm,pm2.5_cf_1,pm10.0_atm,pm10.0_cf_1
0,2025-01-15T07:00:00+06:00,16,72,935.98,NaN,0.04,10.6,16.9,71.6,2470.2,...,112.26,17.30,4.834,2.622,12.0,12.0,18.4,18.4,22.2,22.2
1,2025-01-15T08:00:00+06:00,18,56,936.21,NaN,0.04,15.7,20.3,51.4,3657.7,...,164.92,26.63,6.509,3.412,20.3,19.2,29.3,30.9,36.0,36.5
2,2025-01-15T09:00:00+06:00,22,46,936.45,NaN,0.05,16.4,20.0,52.8,3550.6,...,188.31,33.56,7.445,3.638,20.6,20.3,32.9,33.9,40.7,40.7
3,2025-01-15T10:00:00+06:00,20,50,936.39,NaN,0.05,22.9,22.4,41.6,4652.3,...,275.75,48.06,10.694,5.239,26.8,23.9,39.5,46.1,51.4,55.6
4,2025-01-15T11:00:00+06:00,17,53,936.27,NaN,0.05,14.2,19.3,56.4,3283.3,...,162.32,27.93,5.728,2.579,15.7,15.5,26.4,27.0,31.8,31.9
